# 2 - Simulation théorique

## Importation des modules

In [1]:
# Rechargement des imports
%load_ext autoreload
%autoreload 2

# Importation des modules
# Module de base
# Modules de base
import numpy as np
import pandas as pd
import yaml
import sys

sys.path.append("../../")
sys.path.append("..")

# Chargement du fichier de configurations
with open("../config.yaml") as file:
    config = yaml.safe_load(file)

# Modules ad hoc
from bozio_wasmer_simulations.simulation.theoretical.base import TheoreticalSimulation
from bozio_wasmer_simulations.simulation.theoretical.taux_captation_marginal import (
    CaptationMarginaleSimulator,
)
from bozio_wasmer_simulations.simulation.theoretical.ppv import PPVReintegrationSimulator

## Simulation des profils des cotisations et allègements existants

In [ ]:
# Simulation des cotisations et allègements
# Initialisation
theoretical_simulation = TheoreticalSimulation()
# Itération sur les années de config
data_core_simulation = theoretical_simulation.core_simulation(
    year=config["SIMULATIONS"]["CORE"]["YEAR_SIMUL"],
    simulation_step_smic=config["SIMULATIONS"]["THEORETICAL"]["STEP_SMIC"],
    simulation_max_smic=config["SIMULATIONS"]["THEORETICAL"]["MAX_SMIC"],
)

data_core_simulation.head()

In [ ]:
# Graphique
theoretical_simulation.plot(
    data=data_core_simulation,
    x="assiette_allegement_prop_smic",
    hue=[
        "allegements_prop_assiette",
        "allegement_general_prop_assiette",
        "allegement_cotisation_allocations_familiales_prop_assiette",
        "allegement_cotisation_maladie_prop_assiette",
    ],
    x_label="Salaire (en multiples du SMIC)",
    y_label="Taux",
    hue_label="Allègements",
    labels={
        "allegements_prop_assiette": "Total",
        "allegement_general_prop_assiette": "Allègement général",
        "allegement_cotisation_allocations_familiales_prop_assiette": "Allègement cotisations familiales",
        "allegement_cotisation_maladie_prop_assiette": "Allègement cotisations maladie",
    },
    export_key=None,
    show=True,
)

## Simulation des profils des réformes d'allègements généraux

In [ ]:
# Simulation des réformes des allègements
# Initialisation
reform_simulation = TheoreticalSimulation(
    log_filename="../logs/theoretical_reform_simulation.log"
)
# Itération des réformes
data_reform_simulation = reform_simulation.iterate_reform_simulations(
    scenarios=config["SCENARIOS"],
    year=config["SIMULATIONS"]["CORE"]["YEAR_SIMUL"],
    simulation_step_smic=config["SIMULATIONS"]["THEORETICAL"]["STEP_SMIC"],
    simulation_max_smic=config["SIMULATIONS"]["THEORETICAL"]["MAX_SMIC"],
)

data_reform_simulation.head()

In [ ]:
# Graphique
reform_simulation.plot(
    data=data_reform_simulation,
    x="assiette_allegement_prop_smic",
    hue=["new_allegement_central_prop_assiette", "new_allegement_dss_prop_assiette"],
    x_label="Salaire (en multiples du SMIC)",
    y_label="Taux",
    hue_label="Allègements",
    labels={
        "new_allegement_central_prop_assiette": "Scénario central",
        "new_allegement_dss_prop_assiette": "Scénario DSS",
    },
    export_key=None,
    show=True,
)

## Simulation des taux de captation marginaux associés à chaque scénario

### Construction des taux de captation marginaux pour différents niveaux de rémunération

In [ ]:
# Initialisation du simulateur
captation_simulator = CaptationMarginaleSimulator(project=config['CASD']['PROJET'])
# Itération des simulations
data_captation, data_secret_stat = captation_simulator.build(
    year_data=config["SIMULATIONS"]["CORE"]["YEAR_DATA"],
    year_simul=config["SIMULATIONS"]["CORE"]["YEAR_SIMUL"],
    simulation_step_smic=config["SIMULATIONS"]["THEORETICAL"]["STEP_SMIC"],
    simulation_max_smic=config["SIMULATIONS"]["THEORETICAL"]["MAX_SMIC"],
    scenarios=config["SCENARIOS"],
    data=None,
)
# Ajout de variables spécifiques au scénario "jeunes"
# Création d'une éligibilité au dispositif "jeunes"
captation_simulator.data_dads["ind_jeunes"] = np.where(
    (captation_simulator.data_dads["age"] < 26)
    & (captation_simulator.data_dads["salaire_de_base_prop_smic"] < 1.2),
    "jeunes",
    "vieux",
)
# Itération de la construction des poids sur chacun des populations dans le cas du scénario "jeunes"
list_var_groupby = ["salaire_de_base_prop_smic", "ind_jeunes"]
data_stat_des_jv, data_secret_stat_jv = captation_simulator.build_weights_simulation(
    data_simul=data_captation,
    year=config["SIMULATIONS"]["CORE"]["YEAR_SIMUL"],
    simulation_max_smic=config["SIMULATIONS"]["THEORETICAL"]["MAX_SMIC"],
    list_var_groupby=list_var_groupby,
)
data_stat_des_jv = data_stat_des_jv.unstack()
data_stat_des_jv.columns = [
    "_".join(idx) for idx in data_stat_des_jv.columns.to_flat_index()
]
# Concaténation des deux jeux de données
data_captation = pd.concat(
    [data_captation.set_index("salaire_de_base_prop_smic"), data_stat_des_jv],
    axis=1,
    join="inner",
).reset_index()
# Concaténation des jeux de données de vérification du secret statistique
data_secret_stat = pd.concat(
    [data_secret_stat.set_index("salaire_de_base_prop_smic"), data_secret_stat_jv],
    axis=1,
    join='outer'
)

data_captation.head()

In [ ]:
# Graphique
captation_simulator.plot(
    data=data_captation,
    x="salaire_de_base_prop_smic",
    hue=["taux_captation_marginal", "taux_captation_marginal_central"],
    x_label="Salaire (en multiples du SMIC)",
    y_label="Taux",
    hue_label="Allègements",
    labels={
        "taux_captation_marginal": "Situation actuelle",
        "taux_captation_marginal": "Scénario central",
    },
    export_key=None,
    show=True,
)

### Construction d'indicateurs synthétiques

#### Avec une élasticité de 0.5

In [ ]:
# Construction d'un indicateur synthétique
data_res_05 = captation_simulator.build_taux_synthetique(
    data=data_captation,
    elasticite=config["SIMULATIONS"]["THEORETICAL"]["ELASTICITE_TAUX_MARGINAL"],
    names=list(config["SCENARIOS"].keys()),
    weights=["eqtp_sum", "salaire_de_base_sum"],
)
# Correction du scénario jeunes
for weight in ["eqtp_sum", "salaire_de_base_sum"]:
    data_res_05.loc[weight, "JEUNES"] = (
        0.5
        * (
            data_captation[f"taux_captation_marginal_jeunes"]
            / data_captation["taux_captation_marginal"]
            - 1
        )
        .multiply(other=data_captation[f"{weight}_vieux"])
        .sum()
        / data_captation[weight].sum()
    )
data_res_05.head()

#### Avec une élasticité de 0.25

In [ ]:
# Construction d'un indicateur synthétique
data_res_025 = captation_simulator.build_taux_synthetique(
    data=data_captation,
    elasticite=0.25,
    names=list(config["SCENARIOS"].keys()),
    weights=["eqtp_sum", "salaire_de_base_sum"],
)
# Correction du scénario jeunes
for weight in ["eqtp_sum", "salaire_de_base_sum"]:
    data_res_025.loc[weight, "JEUNES"] = (
        0.25
        * (
            data_captation[f"taux_captation_marginal_jeunes"]
            / data_captation["taux_captation_marginal"]
            - 1
        )
        .multiply(other=data_captation[f"{weight}_vieux"])
        .sum()
        / data_captation[weight].sum()
    )
data_res_025.head()

## Réintégration de la PPV à l'assiette des allègements

### Construction des taux de cotisation implicites pour chacun des scénarios dans le cas d'une PPV éxonérée et réintégrée à l'assiette des allègements généraux

In [ ]:
# Initialisation de la classe
ppv_simulator = PPVReintegrationSimulator()
# Concaténation des résultats des simulations exonérée et réintégrée
data_ppv = pd.concat(
    [
        ppv_simulator.build(
            scenarios=config["SCENARIOS"],
            year=config["SIMULATIONS"]["CORE"]["YEAR_SIMUL"],
            simulation_step_smic=config["SIMULATIONS"]["THEORETICAL"]["STEP_SMIC"],
            simulation_max_smic=config["SIMULATIONS"]["THEORETICAL"]["MAX_SMIC"],
            simulation_case="exoneree",
            ppv=config["SIMULATIONS"]["THEORETICAL"]["PPV"],
        ),
        ppv_simulator.build(
            scenarios=config["SCENARIOS"],
            year=config["SIMULATIONS"]["CORE"]["YEAR_SIMUL"],
            simulation_step_smic=config["SIMULATIONS"]["THEORETICAL"]["STEP_SMIC"],
            simulation_max_smic=config["SIMULATIONS"]["THEORETICAL"]["MAX_SMIC"],
            simulation_case="reintegree",
            ppv=config["SIMULATIONS"]["THEORETICAL"]["PPV"],
        ),
    ],
    axis=1,
)

data_ppv.head()

### Représentation graphique

In [ ]:
# Graphique
ppv_simulator.plot(
    data=data_ppv,
    x="exoneree_salaire_de_base",
    hue=["exoneree_taux_cotisation_implicite", "reintegree_taux_cotisation_implicite"],
    x_label="Salaire (en multiples du SMIC)",
    y_label="Taux",
    hue_label="PPV",
    labels={
        "exoneree_taux_cotisation_implicite": "Soustraite de la PPV à l'assiette des allègements",
        "reintegree_taux_cotisation_implicite": "Réintégration de la PPV à l'assiette des allègements",
    },
    export_key=None,
    show=True,
)